In [1]:
import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('feedback.db')
c = conn.cursor()

# Create a table to store feedback
c.execute('''CREATE TABLE IF NOT EXISTS feedback (
             id INTEGER PRIMARY KEY AUTOINCREMENT,
             outfit_rating INTEGER,
             color_rating INTEGER,
             size_rating INTEGER,
             appearance_rating INTEGER)''')

# Commit the changes and close the connection
conn.commit()
conn.close()


In [4]:
import tkinter as tk
from tkinter import ttk, messagebox
from PIL import Image, ImageTk
import sqlite3
import speech_recognition as sr

class FeedbackForm:
    def __init__(self, root):
        self.root = root
        self.root.title("Feedback Form")
        self.labels = ["Outfit Rating", "Color Rating", "Size Rating", "Overall Appearance Rating"]
        self.ratings = {}

        self.star_images = [ImageTk.PhotoImage(Image.open(f'star{i}.png')) for i in range(1, 6)]
        self.star_empty = ImageTk.PhotoImage(Image.open('star0.png'))

        for i, label in enumerate(self.labels):
            tk.Label(root, text=label).grid(row=i*2, column=0, padx=10, pady=10)
            frame = tk.Frame(root)
            frame.grid(row=i*2, column=1, padx=10, pady=10)
            self.ratings[label] = []
            for j in range(5):
                lbl = tk.Label(frame, image=self.star_empty)
                lbl.pack(side=tk.LEFT)
                lbl.bind('<Button-1>', lambda e, lbl=lbl, index=j, label=label: self.set_rating(lbl, index, label))
                self.ratings[label].append(lbl)

        self.submit_button = tk.Button(root, text="Submit", command=self.submit_feedback)
        self.submit_button.grid(row=len(self.labels)*2, columnspan=2, pady=10)

        self.voice_button = tk.Button(root, text="Voice Input", command=self.voice_input)
        self.voice_button.grid(row=len(self.labels)*2+1, columnspan=2, pady=10)

    def set_rating(self, lbl, index, label):
        for i in range(5):
            self.ratings[label][i].config(image=self.star_empty)
        for i in range(index + 1):
            self.ratings[label][i].config(image=self.star_images[i])

    def get_rating(self, label):
        return sum(1 for star in self.ratings[label] if star.cget('image') != str(self.star_empty))

    def submit_feedback(self):
        ratings = [self.get_rating(label) for label in self.labels]

        if all(ratings):
            conn = sqlite3.connect('feedback.db')
            c = conn.cursor()
            c.execute("INSERT INTO feedback (outfit_rating, color_rating, size_rating, appearance_rating) VALUES (?, ?, ?, ?)",
                      tuple(ratings))
            conn.commit()
            conn.close()
            messagebox.showinfo("Feedback Submitted", "Thank you for your feedback!")
        else:
            messagebox.showerror("Error", "Please provide ratings for all categories.")

    def voice_input(self):
        recognizer = sr.Recognizer()
        mic = sr.Microphone()

        with mic as source:
            recognizer.adjust_for_ambient_noise(source)
            audio = recognizer.listen(source)

        try:
            command = recognizer.recognize_google(audio).lower()
            rating = int(command)
            if 1 <= rating <= 5:
                for label in self.labels:
                    for i in range(5):
                        self.ratings[label][i].config(image=self.star_empty)
                    for i in range(rating):
                        self.ratings[label][i].config(image=self.star_images[i])
            else:
                messagebox.showerror("Error", "Please say a number between 1 and 5.")
        except sr.UnknownValueError:
            messagebox.showerror("Error", "Could not understand the audio.")
        except sr.RequestError:
            messagebox.showerror("Error", "Could not request results from the speech recognition service.")
        except ValueError:
            messagebox.showerror("Error", "Invalid input. Please say a number between 1 and 5.")

# Create the Tkinter application
root = tk.Tk()
app = FeedbackForm(root)
root.mainloop()


ModuleNotFoundError: No module named 'speech_recognition'